In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm
import theano

from scipy.linalg import cholesky

np.random.seed(42)
%matplotlib inline

In [60]:
N = 2000
dim = 2
eps = 0.02

true_coefs = np.array([1, 4])

data = np.random.randn(N, dim)
labels = (true_coefs * data).sum(-1) + eps * np.random.randn(N)

labels

array([-1.50734222, -4.52263528,  4.07029198, ...,  3.70144431,
        0.90530598,  0.15789651])

In [71]:
import pymc3 as pm
import arviz as az


with pm.Model() as model:
    # define priors
    cov = np.array([[16, 0], [0, 16]])
    mu = np.zeros(2)
    coefs = pm.MvNormal('coefs', mu=mu, cov=cov, shape=(2,))
    s = pm.Exponential('error', 1)

    # predictions
    obs = pm.Normal('observation', (coefs * data).sum(-1), s, observed=labels)
    
    # use MCMC to sample
    trace = pm.sample(step=pm.Metropolis([coefs, s]))

/opt/conda/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [error]
>Metropolis: [coefs]
/opt/conda/lib/python3.9/site-packages/pymc3/step_methods/metropolis.py:226: RuntimeWarning: overflow encountered in exp
  "accept": np.exp(accept),


/opt/conda/lib/python3.9/site-packages/pymc3/step_methods/metropolis.py:226: RuntimeWarning: overflow encountered in exp
  "accept": np.exp(accept),
/opt/conda/lib/python3.9/site-packages/pymc3/step_methods/metropolis.py:226: RuntimeWarning: overflow encountered in exp
  "accept": np.exp(accept),


/opt/conda/lib/python3.9/site-packages/pymc3/step_methods/metropolis.py:226: RuntimeWarning: overflow encountered in exp
  "accept": np.exp(accept),
Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 1 seconds.
The number of effective samples is smaller than 10% for some parameters.


In [72]:
trace['coefs'].mean(0), trace['error'].mean()

(array([1.00039266, 4.00024401]), 0.020545901771685457)

In [73]:
pred = (trace['coefs'].mean(0) * data).sum(-1) + trace['error'].mean() * np.random.randn(N)

np.square(pred - labels).mean()

0.0008396578040865955